In [1]:
import os
import random
from dataclasses import dataclass

import numpy as np
import torch
from datasets import load_dataset
from transformers import DecisionTransformerConfig, DecisionTransformerModel, Trainer, TrainingArguments

In [2]:
os.environ["WANDB_DISABLED"] = "true" # we diable weights and biases logging for this tutorial
dataset = load_dataset("edbeeching/decision_transformer_gym_replay", "halfcheetah-expert-v2")


Generating train split: 0 examples [00:00, ? examples/s]

Dataset decision_transformer_gym_replay downloaded and prepared to /Users/emrebaloglu/.cache/huggingface/datasets/edbeeching___decision_transformer_gym_replay/halfcheetah-expert-v2/1.1.0/8a8520881dd09333b04f5a116dd47931314ec44b72597bce8675d40ccd42ad88. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [4]:
dataset.keys()

dict_keys(['train'])

In [6]:
dataset["train"]["observations"]